In [1]:
import os

import numpy as np

In [2]:
folers=os.listdir('20_newsgroups')
dataPath=[]
for folder in folers:
    path=os.path.join('20_newsgroups',folder)
    files=os.listdir(path)
    for file in files:
        filePath=os.path.join(path,file)
        dataPath.append(filePath)

In [3]:
def remove_first_four_lines(path):
    with open(path,'r',encoding='utf-8',errors='ignore') as file:
        lines=file.readlines()
    lines=lines[4:]
    with open(path,'w',encoding='utf-8') as file:
        file.writelines(lines)
for path in dataPath:
    remove_first_four_lines(path)

In [4]:
import random
i=0
trainSet=[]
testSet=[]
temp=[]
while(i<19):
    for j in range(i*1000,(i+1)*1000):
        temp.append(dataPath[j])
    random.shuffle(temp)
    trainSet.append(temp[:500])
    testSet.append(temp[500:])
    temp=[]
    i+=1
i*=1000
while(i<len(dataPath)):
    temp.append(dataPath[i])
    i+=1
random.shuffle(temp)
trainSet.append(temp[:500])
testSet.append(temp[500:])

In [5]:
import os
import math
from collections import defaultdict

# Assuming 'folders' variable is available from the previous code
folders = os.listdir('20_newsgroups')
classes = folders
num_classes = len(classes)

In [6]:

# Initialize variables
word_counts_per_class = [defaultdict(int) for _ in range(num_classes)]
total_words_per_class = [0 for _ in range(num_classes)]
total_docs_per_class = [0 for _ in range(num_classes)]
total_word_counts = defaultdict(int)

# Create a mapping from class name to index
class_to_index = {class_name: index for index, class_name in enumerate(classes)}

# Process training data
for c in range(20):
    for file_path in trainSet[c]:
        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
            text = f.read()
            words = text.strip().split()
            total_docs_per_class[c] += 1
            for word in words:
                word_counts_per_class[c][word] += 1
                total_words_per_class[c] += 1
                total_word_counts[word] += 1

# Identify top 200 most frequent words as stop words
sorted_words = sorted(total_word_counts.items(), key=lambda x: x[1], reverse=True)
stop_words = set([word for word, count in sorted_words[:300]])
len(stop_words)

300

In [7]:
vocabulary = set()
for c in range(num_classes):
    for word in list(word_counts_per_class[c].keys()):
        if word in stop_words:
            total_words_per_class[c] -= word_counts_per_class[c][word]
            del word_counts_per_class[c][word]
        else:
            vocabulary.add(word)
V = len(vocabulary)

In [8]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, KFold

#---------------------------------------
# Previous steps have been done up to V = len(vocabulary) in your code.
# We now have trainSet and testSet as lists of file paths for each class.
# classes and class_to_index variables are also defined.
#---------------------------------------

# Flatten the dataset and create labels
all_docs = []
all_labels = []
for class_idx, docs in enumerate(trainSet):
    for doc_path in docs:
        all_docs.append(doc_path)
        all_labels.append(class_idx)

for class_idx, docs in enumerate(testSet):
    for doc_path in docs:
        all_docs.append(doc_path)
        all_labels.append(class_idx)

In [11]:

# Create a TF-IDF vectorizer
# Since we have already done stop word removal and vocabulary processing,
# we can simply let the vectorizer handle tokenization. If you prefer,
# you can provide custom preprocessed tokens. Here, we rely on raw text.
tfidf = TfidfVectorizer(
    input='filename',   # We'll pass file paths directly
    encoding='utf-8',
    decode_error='ignore',
    lowercase=True,
    stop_words=None,     # We already removed stop words manually, so set to None
    token_pattern=r'\b\w+\b',  # Basic token pattern, can be adjusted
    max_df=1.0,
    min_df=1,
    # If you want to limit vocabulary to the one built previously, you could
    # pass `vocabulary` parameter. For now, we let TF-IDF build it from docs.
)

# Transform the entire dataset into TF-IDF features
X = tfidf.fit_transform(all_docs)
y = np.array(all_labels)

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=42, max_iter=2000)  # 设置迭代上限
X_scaled = StandardScaler(with_mean=False).fit_transform(X)
clf.fit(X_scaled, y)

/Users/harrischen/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(max_iter=2000, random_state=42)

In [13]:
# Implement 5-fold cross-validation
# We use KFold to split the entire dataset into 5 folds.
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [14]:
cv_scores = cross_val_score(clf, X, y, cv=kf, scoring='accuracy')

In [15]:

print("5-fold cross-validation scores: ", cv_scores)
print("Mean accuracy: ", np.mean(cv_scores))


5-fold cross-validation scores:  [0.14575    0.15175    0.14953738 0.15303826 0.14178545]
Mean accuracy:  0.14837221805451364
